In [59]:
import torch
from mmtrack.apis import init_model
from mmcv import VideoReader
from mmengine.dataset import Compose, default_collate
import numpy as np

from mmtrack.utils import register_all_modules
register_all_modules()

In [60]:
model = init_model('../configs/mot/qdtrack/qdtrack_yolox_x.py', device='cpu')

03/20 15:27:58 - mmengine - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
03/20 15:27:58 - mmengine - INFO - Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_x_8x8_300e_coco/yolox_x_8x8_300e_coco_20211126_140254-1ef88d67.pth
03/20 15:27:59 - mmengine - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 320, 1, 1]).
size mismatch for bbox_head.multi_level_conv_cls.0.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([1]).
size mismatch for bbox_head.multi_level_conv_cls.1.weight: copying a param with shape torch.Size([80, 320, 1, 1]) from checkpoint, the shape in current model is torch.Si

/home/hoang/Workspace/mot-mmtrack/mmtrack/apis/inference.py:90: UserWarning: dataset_meta or class names are missed, use None by default.
  warnings.warn('dataset_meta or class names are missed, '


In [74]:
import cv2

def prepare_data(imgs):

    test_pipeline = Compose(model.cfg.test_pipeline[2:])
    data_samples = []
    for frame_id, img in zip(range(len(imgs)), imgs):
        data = dict(
            img=img.astype(np.float32),
            frame_id=frame_id,
            ori_shape=img.shape[:2])
        data = test_pipeline(data)
        data_samples.append(data)
    data_samples = default_collate(data_samples)

    return data_samples

def draw_boxes(boxes, image):
    for i, box in enumerate(boxes):
        color = (0, 0, 0)
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, 1
        )
    return image

In [62]:
imgs = VideoReader('../demo/test1.mp4')
data = prepare_data([imgs[100]])

In [63]:
print(data['inputs']['img'].shape)
img = data['inputs']['img'][:, 0]
fm = model.detector.extract_feat(img)

torch.Size([1, 1, 3, 640, 1152])


In [64]:
model.detector.train_cfg['score_thr'] = 0

In [65]:
losses, proposals = model.detector.bbox_head.loss_and_predict(
    fm, data['data_samples'], model.detector.train_cfg)

In [66]:
proposals[0].bboxes.shape

torch.Size([15120, 4])

In [76]:
rois = proposals[0].bboxes[:1000]
scores = proposals[0].scores[:1000]
rois = rois[scores > 0.01]

scaled_img = data['inputs']['img'].detach().cpu().squeeze().moveaxis(0, -1).int().numpy()
im = draw_boxes(rois, scaled_img)
cv2.imwrite('yolox_proposals.png', im)

True

In [68]:
frcnn = init_model('../configs/mot/qdtrack/qdtrack_faster-rcnn_r50_fpn_8xb2-4e_mot17halftrain_test-mot17halfval.py', device='cpu')

03/20 15:28:10 - mmengine - INFO - load model from: open-mmlab://detectron2/resnet50_caffe
03/20 15:28:10 - mmengine - INFO - Loads checkpoint by openmmlab backend from path: open-mmlab://detectron2/resnet50_caffe
03/20 15:28:11 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: conv1.bias

03/20 15:28:11 - mmengine - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco-person/faster_rcnn_r50_fpn_1x_coco-person_20201216_175929-d022e227.pth
03/20 15:28:11 - mmengine - INFO - Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco-person/faster_rcnn_r50_fpn_1x_coco-person_20201216_175929-d022e227.pth


/home/hoang/Workspace/mot-mmtrack/mmtrack/apis/inference.py:90: UserWarning: dataset_meta or class names are missed, use None by default.
  warnings.warn('dataset_meta or class names are missed, '


In [69]:
frcnn_fm = frcnn.detector.extract_feat(img)
frcnn_proposals = frcnn.detector.rpn_head.predict(
    frcnn_fm, data['data_samples'])

In [70]:
scaled_img = data['inputs']['img'].detach().cpu().squeeze().moveaxis(0, -1).int().numpy()
im = draw_boxes(frcnn_proposals[0].bboxes[:100], scaled_img)
cv2.imwrite('frcnn_proposals.png', im)

True